In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [0]:
# !ls /content/gdrive/My\ Drive/Sentiment\ analysis

In [0]:
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import codecs

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
short_pos = codecs.open("/content/gdrive/My Drive/Sentiment analysis/data/positive.txt","r", encoding='latin2').read()
short_neg = codecs.open("/content/gdrive/My Drive/Sentiment analysis/data/negative.txt","r", encoding='latin2').read()

In [0]:
documents = []
all_words = []

In [0]:
#  j is adject, r is adverb, and v is verb
#allowed_word_types = ["J","R","V"]
allowed_word_types = ["J"]

In [0]:
for p in short_pos.split('\n'):
    documents.append( (p, "pos") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

    
for p in short_neg.split('\n'):
    documents.append( (p, "neg") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())


In [0]:
save_documents = open("/content/gdrive/My Drive/Sentiment analysis/pickles/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()

In [0]:
all_words = nltk.FreqDist(all_words)

In [0]:
word_features = list(all_words.keys())[:5000]

In [0]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [0]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [0]:
save_features = open("featuresets"+".pickle", "wb")
pickle.dump(featuresets, save_features)
save_features.close()

In [0]:
random.shuffle(featuresets)

In [0]:
training_set = featuresets[:10000]
testing_set = featuresets[10000:]

In [0]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
nltk.classify.accuracy(classifier, testing_set)*100

71.08433734939759

In [0]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", nltk.classify.accuracy(MNB_classifier, testing_set)*100)

MNB_classifier accuracy percent: 71.98795180722891


In [0]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", nltk.classify.accuracy(BernoulliNB_classifier, testing_set)*100)

BernoulliNB_classifier accuracy percent: 71.53614457831326


In [0]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

LogisticRegression_classifier accuracy percent: 69.42771084337349


In [0]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

SGDClassifier_classifier accuracy percent: 67.7710843373494


In [0]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

LinearSVC_classifier accuracy percent: 67.01807228915662


In [0]:
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

NuSVC_classifier accuracy percent: 67.16867469879519


In [0]:
all_classifiers = [classifier, 
                   MNB_classifier,
                   BernoulliNB_classifier, 
                   LogisticRegression_classifier, 
                   SGDClassifier_classifier, 
                   LinearSVC_classifier, 
                   NuSVC_classifier]

all_classifier_names = ["classifier", 
                   "MNB_classifier",
                   "BernoulliNB_classifier", 
                   "LogisticRegression_classifier", 
                   "SGDClassifier_classifier", 
                   "LinearSVC_classifier", 
                   "NuSVC_classifier"]

for classifier, classifier_name in zip(all_classifiers, all_classifier_names):
  save_classifier = open("/content/gdrive/My Drive/Sentiment analysis/pickles/"+classifier_name+".pickle", "wb")
  pickle.dump(classifier, save_classifier)
  save_classifier.close()


In [0]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers
    
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
            
        choice_votes = votes.count(mode(votes))
        conf = choice_votes/len(votes)
        return conf

In [0]:
voted_classifier = VoteClassifier(classifier, 
                                  MNB_classifier,
                                  BernoulliNB_classifier, 
                                  LogisticRegression_classifier, 
                                  SGDClassifier_classifier, 
                                  LinearSVC_classifier, 
                                  NuSVC_classifier)

In [0]:
print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)

voted_classifier accuracy percent: 68.67469879518072


In [0]:
def sentiment(text):
  feats = find_features(text)
  return voted_classifier.classify(feats)

In [1]:
sentiment("great movie")

NameError: ignored

In [0]:
#Reading all pickled classifiers
for classifier, classifier_name in zip(all_classifiers, all_classifier_names):
  read_classifier = open("/content/gdrive/My Drive/Sentiment analysis/pickles/"+classifier_name+".pickle", "rb")
  classifier = pickle.load(read_classifier)
  read_classifier.close()